In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install tweet-preprocessor
!pip install transformers
!pip install sentence_transformers
!pip install ranky
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=77c0d31c1ea1420b662355a1f30b4e861ab8b3c41658a29129576491e8d5df19
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e746291

## Imports

In [3]:
import preprocessor as p
import re
import nltk
import numpy as np
import pandas as pd
import transformers
import torch
import string
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
# from py_rankaggregation import CombANZ
import ranky as rk
from keybert import KeyBERT
import spacy
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Preprocessing

In [7]:
# download the necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
# Preprocessing
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Removing URLs, retweets, mentions, stopwords
    rtwords = ['rt', 'RT', 'retweet', 'Retweet', 'RETWEET']
    stops = stopwords.words('english') + rtwords
    
    mess = re.sub(r"http\S+", "", mess)
    mess = re.sub("@[A-Za-z0-9_]+","", mess)
    mess = re.sub("#","", mess)

    # Define a regular expression pattern that matches English words
    pattern = re.compile(r'[a-zA-Z]+')
    
    # Find all matches of the pattern in the text
    matches = re.findall(pattern, mess)
    
    # Join the matches back into a single string
    english_text = ' '.join(matches)
    
    # # STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # # Check characters to see if they are in punctuation
    nopunc = [char for char in english_text if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # # Now just remove any stopwords
    # return ' '.join([word for word in nopunc.split() if word.lower() not in stops])
    
    # define the tokenizer, stemmer, and lemmatizer
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')
    lemmatizer = WordNetLemmatizer()

    # convert to lowercase
    tweet = nopunc.lower()
    # tokenize the tweet
    # tokens = word_tokenize(tweet)
    # remove stop words
    # tokens = [token for token in tokens if token not in stop_words]
    # # stem the tokens
    # tokens = [stemmer.stem(token) for token in tokens]
    # # lemmatize the tokens
    # tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # print("here\n")
    doc = nlp(tweet)
    # doc = [token for token in doc if detect(token) == 'en']
    doc = [token for token in doc if token not in stop_words]
    
    # print("here2\n")
    # Lemmatize the tokens
    lemmas = [token.lemma_ for token in doc]
    
    # print("here3\n")
    # Join the lemmas back into a single string
    processed_tweet = ' '.join(lemmas)

    # print("here4\n")
    # # re-join the tokens into a string
    # processed_tweet = ' '.join(tokens)
    return processed_tweet

## Loading the dataset

In [9]:
olympics_tweets = pd.read_csv('/content/drive/MyDrive/SMAI-project/data/Tokyo_Olympics_tweets.csv')
olympics_tweets = olympics_tweets.iloc[:10000]

In [10]:
olympics_tweets

,id,Tweet,user_screen_name,user_location,retweet_count,favorited,favorite_count,user_description,user_created_at,user_followers,user_friends,date,language
0,1424215952879624194,As Neeraj Chopra throws us into Olympics glory...,nowme_datta,India,0,False,0.0,I bring nothing to the table. I bring the tabl...,2009-07-25 08:28:16,52804.0,1401.0,2021-08-08 03:48:38,en
1,1424215944457461762,@Olympics What is Olympic Truce？？？SHAME,koothles,China,0,False,0.0,NBA Ps4 舞曲/電子,2018-09-15 19:40:23,2.0,44.0,2021-08-08 03:48:36,en
2,1424215943404683264,"@Heisenberg369 Very much, parents want to get ...",Costello_TheRat,India,0,False,0.0,Who's the rat out there?,2011-11-26 18:39:14,502.0,834.0,2021-08-08 03:48:36,en
3,1424215941915631622,HISTORIC! \n\nIndia wins 7 medals at #Tokyo202...,VinodChavdaBJP,"Bhuj, India",0,False,0.0,Member of Parliament (Kachchh) & Advocate । Ge...,2014-01-25 04:28:21,89171.0,1417.0,2021-08-08 03:48:35,en
4,1424215941852778509,Two Superstars of #TeamIndia together.\n\n#his...,crictwig,"Rajasthan, India",0,False,0.0,INDIAN 🇮🇳 ||Cricket Enthusiastic 🏏 ||Sachin×MS...,2020-06-12 10:07:47,1013.0,530.0,2021-08-08 03:48:35,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1424089991991468032,The sport of speed walking has to be my favori...,OhKayKayKay42,"Texas, USA",0,False,0.0,Former professional basketball player| Childre...,2011-03-26 01:22:13,1190.0,646.0,2021-08-07 19:28:07,en
9996,1424089991991468032,The sport of speed walking has to be my favori...,OhKayKayKay42,"Texas, USA",0,False,0.0,Former professional basketball player| Childre...,2011-03-26 01:22:13,1190.0,646.0,2021-08-07 19:28:07,en
9997,1424089988828778498,#2020TokyoOlympics #Entertainment #Sports #Spo...,Algerie_Infos,www.AlgerieInformations.com,0,False,0.0,Pour une République Algérienne Démocratique et...,2013-07-12 20:44:13,4970.0,144.0,2021-08-07 19:28:06,en
9998,1424089988828778498,#2020TokyoOlympics #Entertainment #Sports #Spo...,Algerie_Infos,www.AlgerieInformations.com,0,False,0.0,Pour une République Algérienne Démocratique et...,2013-07-12 20:44:13,4970.0,144.0,2021-08-07 19:28:06,en


In [11]:
olympics_tweets = olympics_tweets[['Tweet']]
olympics_tweets

,Tweet
0,As Neeraj Chopra throws us into Olympics glory...
1,@Olympics What is Olympic Truce？？？SHAME
2,"@Heisenberg369 Very much, parents want to get ..."
3,HISTORIC! \n\nIndia wins 7 medals at #Tokyo202...
4,Two Superstars of #TeamIndia together.\n\n#his...
...,...
9995,The sport of speed walking has to be my favori...
9996,The sport of speed walking has to be my favori...
9997,#2020TokyoOlympics #Entertainment #Sports #Spo...
9998,#2020TokyoOlympics #Entertainment #Sports #Spo...


In [12]:
raw_tweets = olympics_tweets['Tweet'].tolist()
raw_tweets

['As Neeraj Chopra throws us into Olympics glory, do not forget what a wise man said once. https://t.co/Hy54nm238y',
 '@Olympics What is Olympic Truce？？？SHAME',
 '@Heisenberg369 Very much, parents want to get their kids admitted to the best available coaching centre for JEE/NEE… https://t.co/rDqxxK3EcX',
 'HISTORIC! \n\nIndia wins 7 medals at #Tokyo2020\xa0, the highest ever tally in the history of Olympics. \n\nThe entire con… https://t.co/KIUPHRDmx4',
 'Two Superstars of #TeamIndia together.\n\n#history #gold #Athletics #Hockey  #olympics #tokyo https://t.co/Zy0OTcVQzI',
 "BREAKING: Track cyclist Azizulhasni Awang has bagged a silver medal at the Tokyo Olympics, winning Malaysia's secon… https://t.co/axOHSAeENc",
 'So far, the Olympics played Gambler twice and Love Killa once. 😲',
 'caratland and olympics ending ceremony today...I MEAN WHAT ELSE COULD YOU WANT😭💜',
 '@anthonysuntay Yeayyy 🙌🏻🙌🏻🙌🏻 Excited for all of us in the 2024 Olympics ❤️ Galing ng Pinoy 🇵🇭🇵🇭🇵🇭',
 "How did I just le

In [13]:
olympics_tweets['Tweet'] = olympics_tweets['Tweet'].apply(text_process)

<ipython-input-13-9b79568f5e9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olympics_tweets['Tweet'] = olympics_tweets['Tweet'].apply(text_process)


In [14]:
olympics_tweets

,Tweet
0,as neeraj chopra throw we into olympic glory d...
1,what be olympic truce shame
2,very much parent want to get their kid admit t...
3,historic india win medal at tokyo the high eve...
4,two superstar of teamindia together history go...
...,...
9995,the sport of speed walking have to be my favor...
9996,the sport of speed walking have to be my favor...
9997,tokyoolympic entertainment sport sportsmiddlee...
9998,tokyoolympic entertainment sport sportsmiddlee...


In [15]:
tweets = olympics_tweets['Tweet'].tolist()
tweets

['as neeraj chopra throw we into olympic glory do not forget what a wise man say once',
 'what be olympic truce shame',
 'very much parent want to get their kid admit to the good available coach centre for jee nee',
 'historic india win medal at tokyo the high ever tally in the history of olympic the entire con',
 'two superstar of teamindia together history gold athletic hockey olympics tokyo',
 'break track cyclist azizulhasni awang have bag a silver medal at the tokyo olympics win malaysia s secon',
 'so far the olympics play gambler twice and love killa once',
 'caratland and olympic end ceremony today I mean what else could you want',
 'yeayyy excited for all of we in the olympic gale ng pinoy',
 'how do I just learn that megan rapinoe s fianc e be sue bird tokyoolympics olympic',
 'be brittany not allow to dunk in these olympic game',
 'omg malaysia get a silver medal for the olympics yeeeee congrat azizul awang',
 'bhai cricket be a single sport to mai bhi olympics ke single spo

## Get tri-grams from each tweet

In [16]:
def get_ngrams(sentence, n):
    words = sentence.split()
    phrases = []
    for i in range(len(words)-(n-1)):
        phrase = " ".join(words[i:i+n])
        phrases.append(phrase)
    return phrases

In [17]:
# get n-grams for each tweet
ngrams = []
for tweet in tweets:
    ngrams.append(get_ngrams(tweet, 3))

# flatten list of lists
ngrams = [item for sublist in ngrams for item in sublist]
print(len(ngrams))

# get unique n-grams
unique_ngrams = list(set(ngrams))
print(len(unique_ngrams))

124607
82570


## Sentence Transformer Models

In [18]:
# Initializing the sentence transformer models
xlm_dist = SentenceTransformer('sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1')

distilbert = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-tas-b')

para_minilm = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

## Getting the embeddings of tweets and tri-grams

In [19]:
xlm_embeddings = xlm_dist.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nXLM-Roberta embeddings of tweets:")
print(xlm_embeddings)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]


XLM-Roberta embeddings of tweets:
tensor([[ 0.0614, -0.0631,  0.2356,  ...,  0.1537, -0.0406,  0.2665],
        [ 0.0594,  0.0882,  0.1609,  ...,  0.4541,  0.0963,  0.1063],
        [ 0.1501,  0.0767,  0.0192,  ..., -0.0959, -0.1298,  0.1900],
        ...,
        [ 0.0581,  0.3326, -0.0616,  ...,  0.2236, -0.0349,  0.3531],
        [ 0.0581,  0.3326, -0.0616,  ...,  0.2236, -0.0349,  0.3531],
        [ 0.0024,  0.0606,  0.1676,  ...,  0.0872,  0.2070, -0.3109]],
       device='cuda:0')


In [20]:
distilbert_embeddings = distilbert.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nDistilbert embeddings of tweets:")
print(distilbert_embeddings)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]


Distilbert embeddings of tweets:
tensor([[ 0.0112, -0.1390,  0.2219,  ...,  0.0764,  0.0840,  0.0686],
        [-0.1167, -0.2782,  0.0341,  ..., -0.0471, -0.1460,  0.0769],
        [ 0.2261, -0.2023,  0.3353,  ...,  0.2423,  0.2636,  0.1699],
        ...,
        [ 0.0610, -0.1456,  0.1860,  ..., -0.2639,  0.1551, -0.1338],
        [ 0.0610, -0.1456,  0.1860,  ..., -0.2639,  0.1551, -0.1338],
        [ 0.7638, -0.2492, -0.0207,  ..., -0.0254, -0.0071, -0.2023]],
       device='cuda:0')


In [21]:
minilm_embeddings = para_minilm.encode(tweets, convert_to_tensor=True, show_progress_bar=True)
print("\nPara MiniLM embeddings of tweets:")
print(minilm_embeddings)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]


Para MiniLM embeddings of tweets:
tensor([[ 0.4250,  0.3287, -0.2773,  ..., -0.1523,  0.2558,  0.0333],
        [ 0.0865,  0.9522,  0.2686,  ..., -0.5730, -0.3767, -0.1107],
        [ 0.3380, -0.0099, -0.0464,  ...,  0.2384,  0.0416, -0.0057],
        ...,
        [ 0.2778,  0.3291, -0.5351,  ..., -0.5831, -0.2308,  0.3139],
        [ 0.2778,  0.3291, -0.5351,  ..., -0.5831, -0.2308,  0.3139],
        [ 0.1448, -0.0678,  0.0908,  ...,  0.2856, -0.5386, -0.4789]],
       device='cuda:0')


In [22]:
xlm_ngram_embeddings = xlm_dist.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nXLM-Roberta embeddings of n-grams:")
print(xlm_ngram_embeddings)

Batches:   0%|          | 0/2581 [00:00<?, ?it/s]


XLM-Roberta embeddings of n-grams:
tensor([[ 0.4034,  0.4593,  0.0593,  ...,  0.3508,  0.2705,  0.2006],
        [ 0.2368,  0.1143,  0.2936,  ...,  0.4241, -0.1830,  0.0057],
        [ 0.2140,  0.0652,  0.0393,  ...,  0.4829, -0.0427, -0.0508],
        ...,
        [ 0.3088,  0.2379,  0.1549,  ...,  0.5242,  0.3899,  0.2075],
        [-0.1990, -0.1146,  0.2417,  ..., -0.0252, -0.0597, -0.0186],
        [-0.0033,  0.1218,  0.0782,  ...,  0.2316,  0.3393,  0.0462]],
       device='cuda:0')


In [23]:
distilbert_ngram_embeddings = distilbert.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nDistilbert embeddings of n-grams:")
print(distilbert_ngram_embeddings)

Batches:   0%|          | 0/2581 [00:00<?, ?it/s]


Distilbert embeddings of n-grams:
tensor([[ 0.0612,  0.1797, -0.2423,  ...,  0.2053, -0.3946, -0.4487],
        [ 0.2392, -0.0555, -0.0915,  ..., -0.3365, -0.4368, -0.2041],
        [ 0.0329, -0.3389,  0.0588,  ..., -0.1647, -0.1775,  0.0461],
        ...,
        [-0.0362, -0.3043, -0.0252,  ..., -0.1636,  0.0615,  0.1006],
        [ 0.5370, -0.3760,  0.5849,  ..., -0.3455, -0.0151, -0.7136],
        [ 0.3483,  0.0620,  0.0574,  ...,  0.1026, -0.1936, -0.2560]],
       device='cuda:0')


In [24]:
minilm_ngram_embeddings = para_minilm.encode(unique_ngrams, convert_to_tensor=True, show_progress_bar=True)
print("\nPara MiniLM embeddings of n-grams:")
print(minilm_ngram_embeddings)

Batches:   0%|          | 0/2581 [00:00<?, ?it/s]


Para MiniLM embeddings of n-grams:
tensor([[ 0.4650,  0.6671,  0.0137,  ..., -0.2131,  0.5793,  0.4289],
        [-0.2613,  0.4742,  0.2933,  ...,  0.2240, -0.1066,  0.6869],
        [ 0.2195,  0.9176, -0.2526,  ..., -0.1369,  0.8601,  0.5040],
        ...,
        [ 0.3033,  0.4813, -0.0453,  ...,  0.4733, -0.1222,  0.5096],
        [ 0.3689,  0.0166,  0.0116,  ..., -0.3487, -0.0786,  0.0683],
        [-0.4762,  0.8202,  0.3728,  ..., -0.0698,  0.2859,  0.0875]],
       device='cuda:0')


In [63]:
torch.save(xlm_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/olympics_data_embeddings/xlm_tweet_embeddings.pt")
torch.save(xlm_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/olympics_data_embeddings/xlm_ngram_embeddings.pt")
torch.save(distilbert_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/olympics_data_embeddings/distilbert_tweet_embeddings.pt")
torch.save(distilbert_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/olympics_data_embeddings/distilbert_ngram_embeddings.pt")
torch.save(minilm_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/olympics_data_embeddings/minilm_tweet_embeddings.pt")
torch.save(minilm_ngram_embeddings, "/content/drive/MyDrive/SMAI-project/embeddings/olympics_data_embeddings/minilm_ngram_embeddings.pt")

## Computing average Cosine Similarity between each tri-gram and the document

In [26]:
xlm_similarities = []
for ngram in xlm_ngram_embeddings:
    xlm_similarities.append(util.pytorch_cos_sim(xlm_embeddings, ngram).mean().item())

In [27]:
xlm_similarities

[0.14035502076148987,
 0.07551845163106918,
 0.1400044709444046,
 0.19770249724388123,
 0.18209496140480042,
 0.3715619742870331,
 0.09651223570108414,
 0.09618796408176422,
 0.12485487759113312,
 0.1863100528717041,
 0.14783412218093872,
 0.34244394302368164,
 0.11945004761219025,
 0.12105285376310349,
 0.13525857031345367,
 0.03395514562726021,
 0.12187554687261581,
 0.17388944327831268,
 0.11121787130832672,
 0.2719708979129791,
 0.1372816115617752,
 0.10430457442998886,
 0.20489008724689484,
 0.09937702119350433,
 0.30464819073677063,
 0.3783794343471527,
 0.0670509785413742,
 0.13110658526420593,
 0.3311801552772522,
 0.20239956676959991,
 0.2406558096408844,
 0.09165641665458679,
 0.10474514216184616,
 0.12758301198482513,
 0.11837191134691238,
 0.09214531630277634,
 0.101308673620224,
 0.1574600487947464,
 0.16153356432914734,
 0.10365056991577148,
 0.17110846936702728,
 0.0622095949947834,
 0.11481187492609024,
 0.1026599183678627,
 0.08387133479118347,
 0.1882973313331604,
 0.

In [28]:
# minilm_similarity_matrix = util.pytorch_cos_sim(minilm_embeddings, minilm_ngram_embeddings)
# minilm_similarity_matrix

minilm_similarities = []
for ngram in minilm_ngram_embeddings:
    minilm_similarities.append(util.pytorch_cos_sim(minilm_embeddings, ngram).mean().item())

In [29]:
minilm_similarities

[0.14707212150096893,
 0.07724984735250473,
 0.20041558146476746,
 0.25633686780929565,
 0.14006447792053223,
 0.27284881472587585,
 0.04665008932352066,
 0.09490470588207245,
 0.20354658365249634,
 0.29806289076805115,
 0.14683006703853607,
 0.24243012070655823,
 0.17746731638908386,
 0.12362165004014969,
 0.16919615864753723,
 0.03734148293733597,
 0.10394379496574402,
 0.1389491856098175,
 0.13929520547389984,
 0.2905599772930145,
 0.14602971076965332,
 0.11285441368818283,
 0.14228537678718567,
 0.1295330971479416,
 0.3166881799697876,
 0.29062020778656006,
 0.119676373898983,
 0.15287387371063232,
 0.310062438249588,
 0.038964007049798965,
 0.1548183262348175,
 0.15278099477291107,
 0.1304297298192978,
 0.19008615612983704,
 0.1315746158361435,
 0.031909216195344925,
 0.048220716416835785,
 0.11978117376565933,
 0.06553372740745544,
 0.026874542236328125,
 0.13161616027355194,
 0.10900473594665527,
 0.09278206527233124,
 0.09420783817768097,
 0.10638556629419327,
 0.19989643990993

In [30]:
# minilm_similarity_matrix.shape

In [31]:
# # distilbert_similarity_matrix = util.pytorch_cos_sim(distilbert_embeddings, distilbert_ngram_embeddings)
# from sklearn.metrics.pairwise import cosine_similarity
# distilbert_embeddings = distilbert_embeddings.to("cpu")
# distilbert_ngram_embeddings = distilbert_ngram_embeddings.to("cpu")
# distilbert_similarity_matrix = util.pytorch_cos_sim(distilbert_embeddings, distilbert_ngram_embeddings)
# distilbert_similarity_matrix

distilbert_similarities = []
for ngram in distilbert_ngram_embeddings:
    distilbert_similarities.append(util.pytorch_cos_sim(distilbert_embeddings, ngram).mean().item())

In [32]:
# distilbert_similarity_matrix.shape

In [33]:
distilbert_similarities

[0.6273210644721985,
 0.638821005821228,
 0.7424764633178711,
 0.6671527028083801,
 0.7257969379425049,
 0.7494828701019287,
 0.607383668422699,
 0.6735347509384155,
 0.6744714975357056,
 0.7023266553878784,
 0.6558897495269775,
 0.7297154664993286,
 0.6705426573753357,
 0.6791694164276123,
 0.6711490750312805,
 0.5838289856910706,
 0.6796011328697205,
 0.6968389749526978,
 0.673016369342804,
 0.6797083020210266,
 0.6411462426185608,
 0.6761452555656433,
 0.7280117273330688,
 0.6419490575790405,
 0.7299561500549316,
 0.7341450452804565,
 0.6678875088691711,
 0.7001996040344238,
 0.7235026359558105,
 0.7149404883384705,
 0.6741427779197693,
 0.6694133877754211,
 0.6357024312019348,
 0.6794359087944031,
 0.6699368357658386,
 0.6562857031822205,
 0.6678692102432251,
 0.6761693358421326,
 0.6395577192306519,
 0.655333399772644,
 0.6588298678398132,
 0.7042163014411926,
 0.6845768094062805,
 0.6739575862884521,
 0.668613076210022,
 0.6935529112815857,
 0.6343622803688049,
 0.639649271965026

In [34]:
xlm_similarities = np.array(xlm_similarities)
distilbert_similarities = np.array(distilbert_similarities)
minilm_similarities = np.array(minilm_similarities)

## Normailizing the similarities using MinMaxScaler

In [35]:
scaler = MinMaxScaler()
xlm_similarities = scaler.fit_transform(xlm_similarities.reshape(-1, 1))
distilbert_similarities = scaler.fit_transform(distilbert_similarities.reshape(-1, 1))
minilm_similarities = scaler.fit_transform(minilm_similarities.reshape(-1, 1))

## Putting keyphrases and scores in a dataframe and ranking them individually for each model

In [36]:
xlm_scores_df = pd.DataFrame(list(zip(unique_ngrams, xlm_similarities)), columns=['keyphrase', 'score'])
distilbert_scores_df = pd.DataFrame(list(zip(unique_ngrams, distilbert_similarities)), columns=['keyphrase', 'score'])
minilm_scores_df = pd.DataFrame(list(zip(unique_ngrams, minilm_similarities)), columns=['keyphrase', 'score'])

In [37]:
xlm_scores_df

,keyphrase,score
0,most important be,[0.3712066661456081]
1,hold your hand,[0.23688165138346107]
2,it for the,[0.3704804153586467]
3,do it bronze,[0.4900161702323201]
4,damishi sango congratulates,[0.457681293324072]
...,...,...
82565,on olympics oh,[0.9022040106069042]
82566,well but be,[0.30768704351693943]
82567,olympic why someone,[0.6428470583682664]
82568,within next couple,[0.27832102734433]


## Removing keyphrases with scores less than threshold

In [38]:
remove_list = []
threshold = 0.3

anomalies1 = xlm_scores_df[xlm_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies1)

anomalies2 = distilbert_scores_df[distilbert_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies2)

anomalies3 = minilm_scores_df[minilm_scores_df['score'] < threshold]['keyphrase'].tolist()
remove_list.append(anomalies3)

remove_list = [item for sublist in remove_list for item in sublist]
remove_list = list(set(remove_list))

len(remove_list)

30414

In [39]:
# removing the keyphrases from remove_list from all 3 dataframes
xlm_scores_df = xlm_scores_df[~xlm_scores_df['keyphrase'].isin(remove_list)]
distilbert_scores_df = distilbert_scores_df[~distilbert_scores_df['keyphrase'].isin(remove_list)]
minilm_scores_df = minilm_scores_df[~minilm_scores_df['keyphrase'].isin(remove_list)]

In [40]:
xlm_rank_df = xlm_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
xlm_rank_df['rank'] = xlm_rank_df.index + 1
xlm_rank_df

,keyphrase,score,rank
0,neerajchopra goldmedal olympics,[1.0],1
1,olympics neerajchopra goldmedal,[0.9981504298767935],2
2,joy neerajchopra olympic,[0.9919199544751122],3
3,neerajchopra olympics goldmedal,[0.9899062717404928],4
4,olympics medal bajrangpunia,[0.9870807934397126],5
...,...,...,...
52151,almost whole of,[0.30002797647691454],52152
52152,it girl for,[0.30002706576923266],52153
52153,have be really,[0.3000229753025261],52154
52154,cyclingtrack woman s,[0.30000886705131924],52155


In [41]:
distilbert_rank_df = distilbert_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
distilbert_rank_df['rank'] = distilbert_rank_df.index + 1
distilbert_rank_df

,keyphrase,score,rank
0,gold olympic tellmeyourewhitewithoutsayingyour...,[1.0],1
1,neerajgoldchopra olympics tokyo,[0.9897707992843245],2
2,neerajgoldchopra tokyo olympic,[0.9896993802830418],3
3,olympics neerajchopra ravikumardahiya,[0.9878781957503406],4
4,congratulationsneerajchopra tokyo olympics,[0.9841121289747172],5
...,...,...,...
52151,space be great,[0.30006579118398147],52152
52152,s love story,[0.3000569352278224],52153
52153,la go to,[0.30005607819980673],52154
52154,stock s must,[0.3000320814153761],52155


In [42]:
minilm_rank_df = minilm_scores_df.sort_values(by='score', ascending=False).reset_index(drop=True)
minilm_rank_df['rank'] = minilm_rank_df.index + 1
minilm_rank_df

,keyphrase,score,rank
0,olympic not only,[1.0],1
1,this olympic too,[0.9981171884387365],2
2,olympic but this,[0.996992932453415],3
3,anyway olympic be,[0.9949945139575069],4
4,olympic though be,[0.9937283544442416],5
...,...,...,...
52151,msd be in,[0.3001001324008954],52152
52152,I d never,[0.30007264050790067],52153
52153,in the rhythmicgymnastic,[0.3000538298620528],52154
52154,awful it cause,[0.3000406739032256],52155


In [43]:
# temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
# temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
# temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# # create a dataframe with n-gram, average score
# avg_score_df = pd.DataFrame(list(zip(temp1['keyphrase'], (temp1['score'] + temp2['score'] + temp3['score'])/3)), columns=['keyphrase', 'avg_score'])
# avg_score_df

In [44]:
# avg_score_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
# avg_score_df

In [45]:
# Computing threshold

# avg_scores = avg_score_df['avg_score'].tolist()

# dk = 0
# for i in range(len(avg_scores)-1):
#     dk += abs(avg_scores[i+1] - avg_scores[i])

# dk = dk/(len(avg_scores)-1)

# sk = 0
# for i in range(len(avg_scores)-1):
#     sk += abs(abs(avg_scores[i+1] - avg_scores[i]) - dk)

# sk = sk/(len(avg_scores)-1)
# sk = np.sqrt(sk)

# threshold = sk/dk
# threshold = threshold[0]

# print("d(k) = ", dk)
# print("s(k) = ", sk)
# print("Threshold = ", threshold)

In [46]:
# threshold = 0.5

In [47]:
# anomalies = avg_score_df[avg_score_df['avg_score'] < threshold]['keyphrase'].tolist()
# anomalies

In [48]:
# remove these keyphrases from xlm_rank_df, distilbert_rank_df and minilm_rand_df
# xlm_rank_df = xlm_rank_df[~xlm_rank_df['keyphrase'].isin(anomalies)]
# distilbert_rank_df = distilbert_rank_df[~distilbert_rank_df['keyphrase'].isin(anomalies)]
# minilm_rank_df = minilm_rank_df[~minilm_rank_df['keyphrase'].isin(anomalies)]

## Processing to get the array to perform Rank Aggregation

In [49]:
# sort xlm_rank_df, distilbert_rank_df, minilm_rank_df by alphabetical order of n-grams
temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# form a matrix where columns are the keyphrases and three rows, each row representing the rank of the keyphrase in each model
keyphrase_matrix = pd.DataFrame(columns=['xlm_rank', 'distilbert_rank', 'minilm_rank'])
keyphrase_matrix['xlm_rank'] = temp1['rank']
keyphrase_matrix['distilbert_rank'] = temp2['rank']
keyphrase_matrix['minilm_rank'] = temp3['rank']

m = keyphrase_matrix.to_numpy()
m = m.T
m

array([[37674, 26968, 30162, ..., 41937, 18202, 16401],
       [17851, 32716, 22858, ..., 28148,  8583, 21886],
       [28652, 19935, 24550, ..., 43871, 22679, 41714]])

In [50]:
m.shape

(3, 52156)

## Rank Aggregation using `ranky` to get the final rank for each keyphrase

In [51]:
# agg_ranks = rk.kemeny_young(m)
# agg_ranks

## Aggregate Rank
* Rank aggregation using `ranky` didn't work properly. So we just used average scores to give the final ranks.

In [52]:
temp1 = xlm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp2 = distilbert_rank_df.sort_values(by='keyphrase').reset_index(drop=True)
temp3 = minilm_rank_df.sort_values(by='keyphrase').reset_index(drop=True)

# create a dataframe with n-gram, average score
avg_score_df = pd.DataFrame(list(zip(temp1['keyphrase'], (temp1['score'] * 0.45 + temp2['score'] * 0.3 + temp3['score'] * 0.25))), columns=['keyphrase', 'avg_score'])
avg_score_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
avg_score_df

,keyphrase,avg_score
0,neerajchopra goldmedal olympics,[0.9468749834763831]
1,congratulationsneerajchopra tokyo olympics,[0.9451013327756446]
2,olympics neerajchopra goldmedal,[0.9409277047054755]
3,gold olympics neerajchopra,[0.940589651161305]
4,congz neerajgoldchopra olympics,[0.939044678724352]
...,...,...
52151,feel the love,[0.3090460407303004]
52152,self absorb ad,[0.3088443591393485]
52153,people stream in,[0.3075318251281352]
52154,feed I imagine,[0.3062331576299484]


In [53]:
final_rank_df = avg_score_df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)
final_rank_df['rank'] = final_rank_df.index + 1
final_rank_df

,keyphrase,avg_score,rank
0,neerajchopra goldmedal olympics,[0.9468749834763831],1
1,congratulationsneerajchopra tokyo olympics,[0.9451013327756446],2
2,olympics neerajchopra goldmedal,[0.9409277047054755],3
3,gold olympics neerajchopra,[0.940589651161305],4
4,congz neerajgoldchopra olympics,[0.939044678724352],5
...,...,...,...
52151,feel the love,[0.3090460407303004],52152
52152,self absorb ad,[0.3088443591393485],52153
52153,people stream in,[0.3075318251281352],52154
52154,feed I imagine,[0.3062331576299484],52155


In [54]:
final_rank_df.head(20)

,keyphrase,avg_score,rank
0,neerajchopra goldmedal olympics,[0.9468749834763831],1
1,congratulationsneerajchopra tokyo olympics,[0.9451013327756446],2
2,olympics neerajchopra goldmedal,[0.9409277047054755],3
3,gold olympics neerajchopra,[0.940589651161305],4
4,congz neerajgoldchopra olympics,[0.939044678724352],5
5,neerajchopra gold olympics,[0.9311378401093059],6
6,neerajchopra proudindian olympics,[0.9303057259798992],7
7,olympics neerajchopra olympic,[0.929610522603576],8
8,proud olympics neerajgoldchopra,[0.92745091391602],9
9,neerajgoldchopra olympics olympic,[0.9260361007823639],10


## Comparison with `KeyBERT`

In [55]:
concatenated_tweets = ' '.join(raw_tweets)
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(concatenated_tweets, keyphrase_ngram_range=(3, 3), top_n=100)
keywords

[('love olympics neeraj_chopra1', 0.6552),
 ('olympics medalwinners india', 0.6522),
 ('thought olympics neeraj_chopra1', 0.6503),
 ('neeraj_chopra1 sports olympics', 0.65),
 ('olympics neeraj_chopra1 golden', 0.6459),
 ('actual olympics neeraj_chopra1', 0.6443),
 ('neeraj_chopra1 olympics sports', 0.6419),
 ('neeraj_chopra1 olympics gold', 0.6411),
 ('olympics olympics2021 neeraj_chopra1', 0.6382),
 ('neeraj_chopra1 olympics indianolympians', 0.6382),
 ('history olympics neeraj_chopra', 0.6368),
 ('indiaattokyo2020 goldmedal olympics', 0.6348),
 ('olympics thank neeraj_chopra1', 0.6344),
 ('olympics congratulations neeraj_chopra1', 0.6331),
 ('congrats neeraj_chopra1 olympics', 0.633),
 ('whtgqebde4 olympics neeraj_chopra1', 0.6323),
 ('congratulations neeraj_chopra1 olympics', 0.6316),
 ('olympics moments neeraj_chopra1', 0.6302),
 ('neeraj_chopra1 olympics tell', 0.6298),
 ('thank neeraj_chopra1 olympics', 0.6294),
 ('olympics indianolympians jayshah', 0.6293),
 ('olympics jayshah n

## Metrics of the proposed model w.r.t KeyBERT outputs as ground truth

In [56]:
ground_truth = [x[0] for x in keywords]
ground_truth

['love olympics neeraj_chopra1',
 'olympics medalwinners india',
 'thought olympics neeraj_chopra1',
 'neeraj_chopra1 sports olympics',
 'olympics neeraj_chopra1 golden',
 'actual olympics neeraj_chopra1',
 'neeraj_chopra1 olympics sports',
 'neeraj_chopra1 olympics gold',
 'olympics olympics2021 neeraj_chopra1',
 'neeraj_chopra1 olympics indianolympians',
 'history olympics neeraj_chopra',
 'indiaattokyo2020 goldmedal olympics',
 'olympics thank neeraj_chopra1',
 'olympics congratulations neeraj_chopra1',
 'congrats neeraj_chopra1 olympics',
 'whtgqebde4 olympics neeraj_chopra1',
 'congratulations neeraj_chopra1 olympics',
 'olympics moments neeraj_chopra1',
 'neeraj_chopra1 olympics tell',
 'thank neeraj_chopra1 olympics',
 'olympics indianolympians jayshah',
 'olympics jayshah neeraj_chopra1',
 'neeraj_chopra1 olympics heartwarming',
 'congrats olympics neeraj_chopra1',
 'achievement olympics neeraj_chopra1',
 'chopra olympics congrats',
 'neeraj_chopra1 india olympics',
 'neeraj_ch

In [57]:
precision = 0

for keyphrase in final_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        precision += 1

print("Proposed Model Precision: ", precision)

Proposed Model Precision:  2


In [58]:
recall = 0

for keyphrase in ground_truth:
    if keyphrase in final_rank_df.head(100)['keyphrase'].tolist():
        recall += 1

print("Proposed Model Recall: ", recall)

Proposed Model Recall:  2


In [59]:
xlm_precision = 0
for keyphrase in xlm_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        xlm_precision += 1

print("xlm-roberto Precision: ", xlm_precision)

distilbert_precision = 0
for keyphrase in distilbert_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        distilbert_precision += 1

print("Distilbert Precision: ", distilbert_precision)

minilm_precision = 0
for keyphrase in minilm_rank_df.head(100)['keyphrase'].tolist():
    if keyphrase in ground_truth:
        minilm_precision += 1

print("Mini LM Precision: ", minilm_precision)

xlm-roberto Precision:  2
Distilbert Precision:  2
Mini LM Precision:  0


In [60]:
xlm_recall = 0
for keyphrase in ground_truth:
    if keyphrase in xlm_rank_df.head(100)['keyphrase'].tolist():
        xlm_recall += 1

print("xlm-roberto Recall: ", xlm_recall)

distilbert_recall = 0
for keyphrase in ground_truth:
    if keyphrase in distilbert_rank_df.head(100)['keyphrase'].tolist():
        distilbert_recall += 1

print("distilbert Recall: ", distilbert_recall)

minilm_recall = 0
for keyphrase in ground_truth:
    if keyphrase in minilm_rank_df.head(100)['keyphrase'].tolist():
        minilm_recall += 1

print("Mini LM Recall: ", minilm_recall)

xlm-roberto Recall:  2
distilbert Recall:  2
Mini LM Recall:  0


In [61]:
import scipy.stats

def rank_correlation(rank1, rank2):
  
    rho, pval = scipy.stats.spearmanr(rank1, rank2)
    return rho

# model_ranking = ["keyphrase1", "keyphrase2", "keyphrase3"]
# ground_truth = ["keyphrase2", "keyphrase1", "keyphrase4"]
similarity = rank_correlation(final_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of proposed model with KeyBERT: ", similarity)

spearman similarity of proposed model with KeyBERT:  0.0474047404740474


In [62]:
xlm_similarity = rank_correlation(xlm_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of xlm-roberto with KeyBERT: ", xlm_similarity)

distilbert_similarity = rank_correlation(distilbert_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of distilbert model with KeyBERT: ", distilbert_similarity)

minilm_similarity = rank_correlation(minilm_rank_df.head(100)['keyphrase'].tolist(), ground_truth)
print("spearman similarity of Mini LM model with KeyBERT: ", minilm_similarity)

spearman similarity of xlm-roberto with KeyBERT:  -0.10586258625862584
spearman similarity of distilbert model with KeyBERT:  -0.04661266126612661
spearman similarity of Mini LM model with KeyBERT:  0.07809180918091808
